## Goals: Hyper Parameter Optimisation of *QRF* model

This notebook propose different methods of hyper parameter optimisation based on X-Validation :
* Random Search
* Genetic algorithm [Not yet included]

# 1. Data Import and Setup

Imports necessary libraries, sets up environment paths.

In [67]:
# Standard library imports
import os
import sys
import json

# Third-party imports
from functools import partial
import pandas as pd
from quantile_forest import RandomForestQuantileRegressor
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingRegressor

# Append project root to sys.path for local imports
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', '..', '..', '..')))

# Local application imports
from src.utils.model import get_station_stats, custom_log_likelihood
from src.utils.SpatioTemporalSplit import SpatioTemporalSplit
from src.utils.custom_models import SnowIndexComputeTransformer


Defines constants :
* INPUT_DIR must be the same as the one defined in *00 Preprocessing/Feature Engineering*.
* MODEL_DIR is the directory where the exploration models will be saved.

In [68]:
INPUT_DIR = "../../../../data/input/"
MODEL_DIR = "../../../../models/exploration/"

SEED = 42 
ALPHA = 0.1
WEEK_TO_PREDICT=1

DATASET_TRANSFORMS = [
    "rm_gnv_st",
    "pca",
    "snow_index",
    # "oh_enc_date",
    "cyc_enc_date",
    "clust_index",
    "clust_hydro",
    # "scl_feat",
    "scl_feat_wl", # Scale all except waterflow lag
    "rm_wl", # remove custom generated water_flow_lag 3w & 4w ---> Need USE_CUSTOM_PREPROCESS = True
    "slct_ma", # keep only specific mobile average 2w or/and 3w or/and 4w ---> Need USE_CUSTOM_PREPROCESS = True
    "lag_slope" # add an indicator that is calculated between water_flow_lag 1w and 2w 
]

PCA_THRESHOLD = 0.98
N_CLUSTER = 10

DATASET_SPEC = "_".join(DATASET_TRANSFORMS)

if "pca" in DATASET_TRANSFORMS:
    DATASET_SPEC += f"_pct_{PCA_THRESHOLD}"

if "clust_index" in DATASET_TRANSFORMS:
    DATASET_SPEC += f"_geocl_{N_CLUSTER}"

if "clust_hydro" in DATASET_TRANSFORMS:
    DATASET_SPEC += f"_hydcl_{N_CLUSTER}"

DATASET_SPEC = "dataset_custom_23"

# columns to drop : target at different horizon, station_code, and features removed from Feature Selection
TO_DROP = ["water_flow_week1", "water_flow_week2", "water_flow_week3", "water_flow_week4"]

USE_CORRELATED_FEATURES = True

PERMANENT_FEATURES = [
    "latitude",
    "longitude",
    "altitude",
    "catchment",
    "north_hemisphere",
    "month_sin",
    "month_cos",
    "season_sin",
    "season_cos",
    "region_cluster",
    "hydro_cluster",
    # "water_flow_evolve_slope",
]

if USE_CORRELATED_FEATURES: 
    path = f"../../00 Preprocessing/correlation_outputs/strong_correlations_{DATASET_SPEC}.json"

    with open(path, 'r') as file:
        correlated_features = json.load(file)

CORRELATED_FEATURES = correlated_features[f"water_flow_week{WEEK_TO_PREDICT}"]

In [69]:
CORRELATED_FEATURES

['evaporation_lag_1w',
 'evaporation_region',
 'evaporation_region_lag_1w',
 'evaporation_sector_lag_1w',
 'evaporation_sub_sector_lag_1w',
 'evaporation_zone_lag_1w',
 'precipitation_region',
 'precipitation_region_lag_1w',
 'precipitation_sector',
 'precipitation_sector_lag_1w',
 'precipitation_sub_sector',
 'precipitation_sub_sector_lag_1w',
 'precipitation_zone',
 'precipitation_zone_lag_1w',
 'precipitations',
 'precipitations_lag_1w',
 'snow_index',
 'soil_moisture',
 'soil_moisture_region',
 'soil_moisture_sector',
 'soil_moisture_sub_sector',
 'soil_moisture_zone',
 'water_flow_lag_1w',
 'water_flow_lag_2w',
 'water_flow_lag_3w',
 'water_flow_lag_4w',
 'water_flow_ma_2w_lag_1w',
 'water_flow_ma_2w_lag_1w_gauss',
 'water_flow_ma_3w_lag_1w',
 'water_flow_ma_3w_lag_1w_gauss',
 'water_flow_ma_4w_lag_1w',
 'water_flow_ma_4w_lag_1w_gauss']

# 2. Data Loading
Load in the baseline datasets, create the directory to save models.

In [70]:
# load the dataset
ds_train = pd.read_csv(f"{INPUT_DIR}{DATASET_SPEC}.csv")
train_data = ds_train.copy()
train_data.reset_index(inplace=True)
train_data = train_data.loc[:, ~train_data.columns.duplicated()]
ds_train = ds_train.set_index("ObsDate")
y_train = train_data[f"water_flow_week{WEEK_TO_PREDICT}"]
cv_data = train_data.copy()


### 3. Model preparation

Compute station statistics (usefull for scalling)

In [71]:
station_stats = get_station_stats(
    y_train.to_numpy(),
    train_data["station_code"].to_numpy()
)

Create a custom Pipeline to keep track of the station code

In [72]:
from sklearn.feature_selection import SelectKBest, f_regression


cols_to_drop = TO_DROP.copy()
cols_to_drop += ["ObsDate"]
predictor_cols = [col for col in cv_data.columns if col not in cols_to_drop]
preprocessor = ColumnTransformer(transformers=[
    ('select', 'passthrough', predictor_cols)
], remainder='drop')

snowIndexer = SnowIndexComputeTransformer()

qrf_week1 = RandomForestQuantileRegressor(n_estimators=10, max_depth=10, min_samples_leaf=10)
# qrf_week1 = GradientBoostingRegressor()

pipeline = Pipeline(steps=[
    # ('snowindexer', SnowIndexComputeTransformer(temp_col_name="tempartures_pca_1", rain_col_name="precipitations_pca_1",)),
    ('preprocessor', preprocessor),
    ('model', qrf_week1)
])

Initialisation of the log likelihood scorer

In [73]:
def inverted_log_likelihood(estimator, X, y_true, cv_data, station_stats, alpha=0.1):
    return -custom_log_likelihood(estimator, X, y_true, cv_data, station_stats, alpha=alpha)

In [74]:
scorer = partial(inverted_log_likelihood,
                 cv_data=cv_data,
                 station_stats=station_stats,
                 alpha=ALPHA)

Initialisation of the SpatioTemporal Splitter

In [75]:
cv = SpatioTemporalSplit(
    n_splits=10,
    date_col='ObsDate',
    station_col='station_code',
    temporal_frac=0.75,
    spatial_frac=0.75,
    random_state=42
)


### 4. Hyper parameter tuning

Define the hyperparameter distributions for random search, take care the parameters presented here are choosen so that the search is fast you need to explore wider parameters range.

#### a. Random Search

In [76]:
# param_distributions = {
#     'model__n_estimators': [2, 10, 20, 45, 60, 85, 100],
#     'model__max_depth': [2, 7, 13, 20, 30, 50],
#     'model__min_samples_leaf': [1, 4, 9, 15, 20, 30],
#     'model__min_samples_split': [2, 5, 10, 20],
#     'model__max_features': ['sqrt', 'log2', 0.3, 0.7, None],
#     'model__bootstrap': [True, False],
#     'snowindexer__altitude_weight': [0.9, 1, 1.1, 1.2],
#     'snowindexer__temp_weight': [0.9, 1, 1.1, 1.2],
#     'snowindexer__precip_weight': [0.1, 0.15, 0.2, 0.25],
# }

# 'model__n_estimators': 35, 'model__min_samples_split': 9, 'model__min_samples_leaf': 25, 'model__max_features': None, 'model__max_depth': 50, 'model__bootstrap': True
# param_distributions = {
#     'model__n_estimators': [20, 25, 30, 35, 40, 45, 50],
#     'model__max_depth': [35, 40, 45, 50, 55, 60, 65],
#     'model__min_samples_leaf': [20, 23, 25, 28, 31],
#     'model__min_samples_split': [6, 7, 8, 9, 10, 11, 12],
#     'model__max_features': [None],
#     'model__bootstrap': [True],
# }

# param_distributions = {
#     'model__n_estimators': [25, 29, 32, 35, 38, 41, 45, 50],
#     'model__max_depth': [40, 43, 46, 48, 50, 52, 55, 59],
#     'model__min_samples_leaf': [15, 18, 22,  25, 27, 30, 32],
#     'model__min_samples_split': [5, 7, 8, 9, 11, 13, 15],
#     'model__max_features': [None],
#     'model__bootstrap': [True]
# }

import numpy as np

param_distributions = {
    'model__n_estimators': [5, 15, 35, 55, 85, 100, 130, 150, 190],
    'model__max_depth': [2, 7, 13, 20, 30, 50, 70, 100],
    'model__min_samples_leaf': [9, 15, 25, 35, 50, 60, 80, 105],
    'model__min_samples_split': [9, 15, 25, 35, 50, 60, 80, 105],
    'model__max_features': ['sqrt', 'log2', 0.3, 0.7, None],
    'model__bootstrap': [True, False]
}

# 9. Set up RandomizedSearchCV.
random_search = RandomizedSearchCV(
    estimator=pipeline,
    param_distributions=param_distributions,
    n_iter=35,            # Number of parameter settings sampled
    scoring=scorer,       # Use our custom scorer
    cv=cv,                # Our custom spatio-temporal splitter
    random_state=42,
    n_jobs=-1,             # Use all available cores
    verbose=3,
    error_score='raise'
)
if USE_CORRELATED_FEATURES:
    print("Using correlated features")
    # cv_data = cv_data[PERMANENT_FEATURES + CORRELATED_FEATURES + ["station_code", "ObsDate", "index"]]

random_search.fit(cv_data[PERMANENT_FEATURES + CORRELATED_FEATURES + ["station_code", "ObsDate"]], y_train)

Using correlated features
Fitting 10 folds for each of 35 candidates, totalling 350 fits


ValueError: A given column is not a column of the dataframe

In [ ]:
cv_data[PERMANENT_FEATURES + CORRELATED_FEATURES + ["station_code", "ObsDate", "index"]]

latitude  longitude  altitude  catchment  north_hemisphere  \
0      43.187673  -0.597133     224.0      488.0                 1   
1      43.187673  -0.597133     224.0      488.0                 1   
2      43.187673  -0.597133     224.0      488.0                 1   
3      43.187673  -0.597133     224.0      488.0                 1   
4      43.187673  -0.597133     224.0      488.0                 1   
...          ...        ...       ...        ...               ...   
27697 -19.533300 -40.629700      31.0    76400.0                 0   
27698 -19.533300 -40.629700      31.0    76400.0                 0   
27699 -19.533300 -40.629700      31.0    76400.0                 0   
27700 -19.533300 -40.629700      31.0    76400.0                 0   
27701 -19.533300 -40.629700      31.0    76400.0                 0   

          month_sin  month_cos  season_sin    season_cos  region_cluster  ...  \
0      5.000000e-01   0.866025         1.0  6.123234e-17               0  ...   
1      5.000000e-01   0.866025         1.0  6.123234e-17               0  ...   
2      8.660254e-01   0.500000         1.0  6.123234e-17               0  ...   
3      8.660254e-01   0.500000         1.0  6.123234e-17               0  ...   
4      8.660254e-01   0.500000         1.0  6.123234e-17               0  ...   
...             ...        ...         ...           ...             ...  ...   
27697 -2.449294e-16   1.000000        -1.0 -1.836970e-16               7  ...   
27698 -2.449294e-16   1.000000        -1.0 -1.836970e-16               7  ...   
27699 -2.449294e-16   1.000000        -1.0 -1.836970e-16               7  ...   
27700 -2.449294e-16   1.000000        -1.0 -1.836970e-16               7  ...   
27701  5.000000e-01   0.866025        -1.0 -1.836970e-16               7  ...   

       water_flow_lag_4w  water_flow_ma_2w_lag_1w  \
0              78.724926                 9.319286   
1              78.724926                 9.110714   
2               8.567143                 8.171429   
3              10.071429                 9.117143   
4               8.150000                10.034286   
...                  ...                      ...   
27697         418.903129               464.509443   
27698         439.257043               462.085421   
27699         412.295671               916.479836   
27700         516.723214              1024.574700   
27701         407.447629               617.894707   

       water_flow_ma_2w_lag_1w_gauss  water_flow_ma_3w_lag_1w  \
0                           9.319286                 9.319286   
1                           9.110714                 8.929524   
2                           8.171429                 8.804762   
3                           9.117143                 8.794762   
4                          10.034286                 9.420476   
...                              ...                      ...   
27697                     464.509443               456.091976   
27698                     462.085421               445.488838   
27699                     916.479836               783.227629   
27700                    1024.574700               818.865676   
27701                     617.894707               887.100486   

       water_flow_ma_3w_lag_1w_gauss  water_flow_ma_4w_lag_1w  \
0                          78.746370                 9.319286   
1                           8.929524                 8.929524   
2                           8.804762                 8.745357   
3                           8.794762                 9.113929   
4                           9.420476                 9.102857   
...                              ...                      ...   
27697                     456.091976               446.794764   
27698                     445.488838               443.930889   
27699                     783.227629               690.494639   
27700                     818.865676               743.330061   
27701                     887.100486               767.187271

In [ ]:
gdfxc

NameError: name 'gdfxc' is not defined

In [ ]:
#Dataset 21 0.75
print("Best Parameters:", random_search.best_params_)
print("Best Score:", random_search.best_score_)

Best Parameters: {'model__n_estimators': 35, 'model__min_samples_split': 9, 'model__min_samples_leaf': 25, 'model__max_features': None, 'model__max_depth': 50, 'model__bootstrap': True}
Best Score: -1.8690944956728


In [ ]:
#Dataset 18 0.5
print("Best Parameters:", random_search.best_params_)
print("Best Score:", random_search.best_score_)

Best Parameters: {'model__n_estimators': 100, 'model__min_samples_split': 50, 'model__min_samples_leaf': 25, 'model__max_features': None, 'model__max_depth': 70, 'model__bootstrap': True}
Best Score: -1.903410350380548


In [ ]:
#Dataset 19 0.75
print("Best Parameters:", random_search.best_params_)
print("Best Score:", random_search.best_score_)

Best Parameters: {'model__n_estimators': 35, 'model__min_samples_split': 9, 'model__min_samples_leaf': 25, 'model__max_features': None, 'model__max_depth': 50, 'model__bootstrap': True}
Best Score: -1.9227578910463272


In [ ]:
#dataset 18 0.75
print("Best Parameters:", random_search.best_params_)
print("Best Score:", random_search.best_score_)

Best Parameters: {'model__n_estimators': 25, 'model__min_samples_split': 11, 'model__min_samples_leaf': 27, 'model__max_features': None, 'model__max_depth': 48, 'model__bootstrap': True}
Best Score: -1.8712027049753694


In [ ]:
print("Best Parameters:", random_search.best_params_)
print("Best Score:", random_search.best_score_)

Best Parameters: {'model__n_estimators': 35, 'model__min_samples_split': 9, 'model__min_samples_leaf': 25, 'model__max_features': None, 'model__max_depth': 50, 'model__bootstrap': True}
Best Score: -1.87843879114098


In [ ]:
print("Best Parameters:", random_search.best_params_)
print("Best Score:", random_search.best_score_)

Best Parameters: {'model__n_estimators': 35, 'model__min_samples_split': 9, 'model__min_samples_leaf': 25, 'model__max_features': None, 'model__max_depth': 50, 'model__bootstrap': True}
Best Score: -1.8851951993485456


In [ ]:
print("Best Parameters:", random_search.best_params_)
print("Best Score:", random_search.best_score_)

Best Parameters: {'model__n_estimators': 35, 'model__min_samples_split': 9, 'model__min_samples_leaf': 25, 'model__max_features': None, 'model__max_depth': 50, 'model__bootstrap': True}
Best Score: -1.8790275140926007


In [ ]:
print("Best Parameters:", random_search.best_params_)
print("Best Score:", random_search.best_score_)

Best Parameters: {'model__n_estimators': 35, 'model__min_samples_split': 50, 'model__min_samples_leaf': 15, 'model__max_features': 0.7, 'model__max_depth': 70, 'model__bootstrap': True}
Best Score: -1.938040627386244


### params for config : 

```python
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ("selector", SelectKBest(score_func=f_regression)),
    ('model', qrf_week1)
])


DATASET_TRANSFORMS = [
    "rm_gnv_st",
    "pca",
    "snow_index",
    # "oh_enc_date",
    "cyc_enc_date",
    "clust_index",
    "clust_hydro",
    # "scl_feat",
    "scl_feat_wl", # Scale all except waterflow lag
    "rm_wl", # remove custom generated water_flow_lag 3w & 4w ---> Need USE_CUSTOM_PREPROCESS = True
    "slct_ma", # keep only specific mobile average 2w or/and 3w or/and 4w ---> Need USE_CUSTOM_PREPROCESS = True
    "lag_slope" # add an indicator that is calculated between water_flow_lag 1w and 2w 
]

PCA_THRESHOLD = 0.98
N_CLUSTER = 10
SELECTED_MOBILE_AVERAGE = [
    # "water_flow_ma_4w_lag_1w",
    # "water_flow_ma_3w_lag_1w",
    # "water_flow_ma_2w_lag_1w",
    "water_flow_ma_4w_lag_1w_gauss",
    # "water_flow_ma_3w_lag_1w_gauss",
    # "water_flow_ma_2w_lag_1w_gauss"
]

```


In [ ]:
print("Best Parameters:", random_search.best_params_)
print("Best Score:", random_search.best_score_)

Best Parameters: {'selector__k': np.int64(27), 'model__n_estimators': 85, 'model__min_samples_split': 80, 'model__min_samples_leaf': 15, 'model__max_features': None, 'model__max_depth': 13, 'model__bootstrap': True}
Best Score: -1.9577376316109347


In [ ]:
best_selector = random_search.best_estimator_.named_steps["selector"]

selected_mask = best_selector.get_support()
adjusted_data = cv_data.drop(columns=cols_to_drop)
selected_features = adjusted_data.columns[selected_mask]

print("Selected features:")
print(selected_features.tolist())
with open(f"{DATASET_SPEC}_selected_features_week{WEEK_TO_PREDICT}.json", 'w') as json_file:
    json.dump(selected_features.tolist(), json_file, indent=4)


Selected features:
['index', 'precipitations_lag_1w_pca_2', 'precipitations_pca_1', 'precipitations_pca_2', 'tempartures_lag_1w_pca_1', 'tempartures_pca_1', 'tempartures_pca_2', 'soil_moisture_pca_1', 'soil_moisture_pca_2', 'soil_moisture_pca_3', 'evaporation_lag_1w_pca_1', 'evaporation_pca_1', 'soil_composition_pca_1', 'soil_composition_pca_4', 'soil_composition_pca_6', 'soil_composition_pca_7', 'soil_composition_pca_9', 'latitude', 'longitude', 'catchment', 'altitude', 'water_flow_lag_1w', 'water_flow_lag_2w', 'water_flow_ma_4w_lag_1w_gauss', 'north_hemisphere', 'snow_index', 'month_cos']


### params for config : 

```python
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', qrf_week1)
])


DATASET_TRANSFORMS = [
    "rm_gnv_st",
    "pca",
    "snow_index",
    # "oh_enc_date",
    "cyc_enc_date",
    "clust_index",
    "clust_hydro",
    "scl_feat",
    # "scl_feat_wl", # Scale all except waterflow lag
    "rm_wl", # remove custom generated water_flow_lag 3w & 4w ---> Need USE_CUSTOM_PREPROCESS = True
    "slct_ma", # keep only specific mobile average 2w or/and 3w or/and 4w ---> Need USE_CUSTOM_PREPROCESS = True
    "lag_slope" # add an indicator that is calculated between water_flow_lag 1w and 2w 
]

PCA_THRESHOLD = 0.98
N_CLUSTER = 10
```


In [ ]:
print("Best Parameters:", random_search.best_params_)
print("Best Score:", random_search.best_score_)

Best Parameters: {'model__n_estimators': 35, 'model__min_samples_split': 9, 'model__min_samples_leaf': 25, 'model__max_features': None, 'model__max_depth': 50, 'model__bootstrap': True}
Best Score: -1.8825202404996986


### params for config : 

```python
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', qrf_week1)
])


DATASET_TRANSFORMS = [
    "rm_gnv_st",
    "pca",
    "snow_index",
    # "oh_enc_date",
    "cyc_enc_date",
    "clust_index",
    "clust_hydro",
    "scl_feat",
    # "scl_feat_wl", # Scale all except waterflow lag
    "rm_wl", # remove custom generated water_flow_lag 3w & 4w ---> Need USE_CUSTOM_PREPROCESS = True
    "slct_ma", # keep only specific mobile average 2w or/and 3w or/and 4w ---> Need USE_CUSTOM_PREPROCESS = True
    "lag_slope" # add an indicator that is calculated between water_flow_lag 1w and 2w 
]

PCA_THRESHOLD = 0.98
N_CLUSTER = 10
```


In [ ]:
print("Best Parameters:", random_search.best_params_)
print("Best Score:", random_search.best_score_)

Best Parameters: {'model__n_estimators': 100, 'model__min_samples_split': 50, 'model__min_samples_leaf': 25, 'model__max_features': None, 'model__max_depth': 70, 'model__bootstrap': True}
Best Score: -1.9117332654881167


### params for config : 

```python
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', qrf_week1)
])


DATASET_TRANSFORMS = [
    "rm_gnv_st",
    "pca",
    "snow_index",
    # "oh_enc_date",
    "cyc_enc_date",
    "clust_index",
    "clust_hydro",
    # "scl_feat",
    "scl_feat_wl", # Scale all except waterflow lag
    "rm_wl", # remove custom generated water_flow_lag 3w & 4w ---> Need USE_CUSTOM_PREPROCESS = True
    "slct_ma", # keep only specific mobile average 2w or/and 3w or/and 4w ---> Need USE_CUSTOM_PREPROCESS = True
    "lag_slope" # add an indicator that is calculated between water_flow_lag 1w and 2w 
]

PCA_THRESHOLD = 0.98
N_CLUSTER = 10
```


In [ ]:
print("Best Parameters:", random_search.best_params_)
print("Best Score:", random_search.best_score_)

Best Parameters: {'model__n_estimators': 35, 'model__min_samples_split': 11, 'model__min_samples_leaf': 23, 'model__max_features': None, 'model__max_depth': 40, 'model__bootstrap': True}
Best Score: -1.8653183950465695


### params for config : 

```python
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', qrf_week1)
])


DATASET_TRANSFORMS = [
    "rm_gnv_st",
    "pca",
    "snow_index",
    # "oh_enc_date",
    "cyc_enc_date",
    "clust_index",
    "clust_hydro",
    # "scl_feat",
    "scl_feat_wl", # Scale all except waterflow lag
    "rm_wl", # remove custom generated water_flow_lag 3w & 4w ---> Need USE_CUSTOM_PREPROCESS = True
    "slct_ma", # keep only specific mobile average 2w or/and 3w or/and 4w ---> Need USE_CUSTOM_PREPROCESS = True
    "lag_slope" # add an indicator that is calculated between water_flow_lag 1w and 2w 
]

PCA_THRESHOLD = 0.98
N_CLUSTER = 10
```


In [ ]:
print("Best Parameters:", random_search.best_params_)
print("Best Score:", random_search.best_score_)

Best Parameters: {'model__n_estimators': 35, 'model__min_samples_split': 9, 'model__min_samples_leaf': 25, 'model__max_features': None, 'model__max_depth': 50, 'model__bootstrap': True}
Best Score: -1.8821667848494648


### params for config : 

```python
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', qrf_week1)
])


DATASET_TRANSFORMS = [
    "rm_gnv_st",
    "pca",
    "snow_index",
    # "oh_enc_date",
    "cyc_enc_date",
    "clust_index",
    "clust_hydro",
    # "scl_feat",
    "scl_feat_wl", # Scale all except waterflow lag
    # "rm_wl", # remove custom generated water_flow_lag 3w & 4w
    # "rm_ma", # remove custom generated mobile average 2w & 3w & 4w
    "lag_slope" # add an indicator that is calculated between water_flow_lag 1w and 2w 
]

PCA_THRESHOLD = 0.98
N_CLUSTER = 10
```


In [ ]:
print("Best Parameters:", random_search.best_params_)
print("Best Score:", random_search.best_score_)

Best Parameters: {'model__n_estimators': 160, 'model__min_samples_split': 66, 'model__min_samples_leaf': 42, 'model__max_features': None, 'model__max_depth': 25, 'model__bootstrap': True}
Best Score: -1.9426425655553738


### params for config : 

```python
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', qrf_week1)
])

DATASET_TRANSFORMS = [
    "rm_gnv_st",
    "pca",
    "snow_index",
    # "oh_enc_date",
    "cyc_enc_date",
    "clust_index",
    "clust_hydro",
    # "scl_feat",
    "scl_feat_wl", # Scale all except waterflow lag
]

PCA_THRESHOLD = 0.98
N_CLUSTER = 5
```


In [ ]:
print("Best Parameters:", random_search.best_params_)
print("Best Score:", random_search.best_score_)

Best Parameters: {'model__n_estimators': 150, 'model__min_samples_split': 30, 'model__min_samples_leaf': 18, 'model__max_features': None, 'model__max_depth': 23, 'model__bootstrap': True}
Best Score: -1.9362015137618989


### params for config : 

```python
pipeline = Pipeline(steps=[
    ('snowindexer', SnowIndexComputeTransformer(temp_col_name="tempartures_pca_1", rain_col_name="precipitations_pca_1",)),
    ('preprocessor', preprocessor),
    ('model', qrf_week1)
])

DATASET_TRANSFORMS = [
    "rm_gnv_st",
    "pca",
    # "snow_index",
    # "oh_enc_date",
    "cyc_enc_date",
    "clust_index",
    "clust_hydro",
    # "scl_feat",
    "scl_feat_wl", # Scale all except waterflow lag
]
```


In [ ]:
print("Best Parameters:", random_search.best_params_)
print("Best Score:", random_search.best_score_)

Best Parameters: {'snowindexer__temp_weight': 1.2, 'snowindexer__precip_weight': 0.15, 'snowindexer__altitude_weight': 0.9, 'model__n_estimators': 100, 'model__min_samples_split': 5, 'model__min_samples_leaf': 30, 'model__max_features': None, 'model__max_depth': 50, 'model__bootstrap': False}
Best Score: -2.1058580480136575


### params for config : 

```python
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', qrf_week1)
])

DATASET_TRANSFORMS = [
    "rm_gnv_st",
    "pca",
    "snow_index",
    "oh_enc_date",
    # "cyc_enc_date",
    # "clust_index",
    "clust_hydro",
    # "scl_feat",
    "scl_feat_wl", # Scale all except waterflow lag
]
```


In [ ]:
print("Best Parameters:", random_search.best_params_)
print("Best Score:", random_search.best_score_)

Best Parameters: {'model__n_estimators': 100, 'model__min_samples_split': 10, 'model__min_samples_leaf': 40, 'model__max_features': None, 'model__max_depth': 13, 'model__bootstrap': True}
Best Score: -1.9542643097106727


### params for config : 

```python
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', qrf_week1)
])


DATASET_TRANSFORMS = [
    "rm_gnv_st",
    "pca",
    "snow_index",
    "oh_enc_date",
    # "cyc_enc_date",
    "clust_index",
    "clust_hydro",
    "scl_feat",
    # "scl_feat_wl", # Scale all except waterflow lag
]
```


In [ ]:
print("Best Parameters:", random_search.best_params_)
print("Best Score:", random_search.best_score_)

Best Parameters: {'model__n_estimators': 45, 'model__min_samples_split': 40, 'model__min_samples_leaf': 20, 'model__max_features': 0.7, 'model__max_depth': 30, 'model__bootstrap': False}
Best Score: -1.9515434055944696


### params for config : 

```python
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', qrf_week1)
])


DATASET_TRANSFORMS = [
    "rm_gnv_st",
    "pca",
    "snow_index",
    # "oh_enc_date",
    "cyc_enc_date",
    "clust_index",
    "clust_hydro",
    "scl_feat",
    # "scl_feat_wl", # Scale all except waterflow lag
]
```


In [ ]:
print("Best Parameters:", random_search.best_params_)
print("Best Score:", random_search.best_score_)

Best Parameters: {'model__n_estimators': 130, 'model__min_samples_split': 30, 'model__min_samples_leaf': 20, 'model__max_features': None, 'model__max_depth': 30, 'model__bootstrap': True}
Best Score: -1.935871280814628


### params for config : 

```python
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', qrf_week1)
])


DATASET_TRANSFORMS = [
    "rm_gnv_st",
    "pca",
    "snow_index",
    # "oh_enc_date",
    "cyc_enc_date",
    "clust_index",
    "clust_hydro",
    # "scl_feat",
    "scl_feat_wl", # Scale all except waterflow lag
    "scl_catch",
]
```


In [ ]:
print("Best Parameters:", random_search.best_params_)
print("Best Score:", random_search.best_score_)

Best Parameters: {'model__n_estimators': 135, 'model__min_samples_split': 27, 'model__min_samples_leaf': 27, 'model__max_features': None, 'model__max_depth': 30, 'model__bootstrap': True}
Best Score: -1.9339345588124381


### params for config : 

```python
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', qrf_week1)
])


DATASET_TRANSFORMS = [
    "rm_gnv_st",
    "pca",
    "snow_index",
    # "oh_enc_date",
    "cyc_enc_date",
    "clust_index",
    "clust_hydro",
    # "scl_feat",
    "scl_feat_wl", # Scale all except waterflow lag
    "scl_catch",
]
```


In [ ]:
print("Best Parameters:", random_search.best_params_)
print("Best Score:", random_search.best_score_)

Best Parameters: {'model__n_estimators': 130, 'model__min_samples_split': 30, 'model__min_samples_leaf': 20, 'model__max_features': None, 'model__max_depth': 30, 'model__bootstrap': True}
Best Score: -1.9395931333152727


### params for config : 

```python
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', qrf_week1)
])

DATASET_TRANSFORMS = [
    "rm_gnv_st",
    "pca",
    "snow_index",
    # "oh_enc_date",
    "cyc_enc_date",
    "clust_index",
    "scl_feat",
    # "scl_feat_wl", # Scale all except waterflow lag
    "scl_catch",
]
```


In [ ]:
print("Best Parameters:", random_search.best_params_)
print("Best Score:", random_search.best_score_)

Best Parameters: {'model__n_estimators': 23, 'model__min_samples_split': 15, 'model__min_samples_leaf': 21, 'model__max_features': 0.7, 'model__max_depth': 23, 'model__bootstrap': True}
Best Score: -1.9359472748030164


### params for config : 

```python
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', qrf_week1)
])

DATASET_TRANSFORMS = [
    "rm_gnv_st",
    "pca",
    "snow_index",
    # "oh_enc_date",
    "cyc_enc_date",
    "clust_index",
    "scl_feat",
    # "scl_feat_wl", # Scale all except waterflow lag
    "scl_catch",
]
```


In [ ]:
print("Best Parameters:", random_search.best_params_)
print("Best Score:", random_search.best_score_)

Best Parameters: {'model__n_estimators': 20, 'model__min_samples_split': 10, 'model__min_samples_leaf': 20, 'model__max_features': 0.7, 'model__max_depth': 20, 'model__bootstrap': True}
Best Score: -1.95292479637325


### params for config : 

```python
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', qrf_week1)
])

DATASET_TRANSFORMS = [
    "rm_gnv_st",
    "pca",
    "snow_index",
    # "oh_enc_date",
    "cyc_enc_date",
    "clust_index",
    "scl_feat",
    # "scl_feat_wl", # Scale all except waterflow lag
    "scl_catch",
]
```


In [ ]:
print("Best Parameters:", random_search.best_params_)
print("Best Score:", random_search.best_score_)

Best Parameters: {'model__n_estimators': 50, 'model__min_samples_split': 18, 'model__min_samples_leaf': 16, 'model__max_features': None, 'model__max_depth': 16, 'model__bootstrap': True}
Best Score: -1.9292546040045893


### params for config : 

```python
pipeline = Pipeline(steps=[
    ('snowindexer', SnowIndexComputeTransformer(temp_col_name="tempartures_pca_1", rain_col_name="precipitations_pca_1",)),
    ('preprocessor', preprocessor),
    ('model', qrf_week1)
])

DATASET_TRANSFORMS = [
    "rm_gnv_st",
    "pca",
    "snow_index",
    # "oh_enc_date",
    "cyc_enc_date",
    "clust_index",
    "scl_feat",
    # "scl_feat_wl", # Scale all except waterflow lag
    "scl_catch",
]
```


In [ ]:
print("Best Parameters:", random_search.best_params_)
print("Best Score:", random_search.best_score_)

Best Parameters: {'model__n_estimators': 60, 'model__min_samples_split': 20, 'model__min_samples_leaf': 9, 'model__max_features': None, 'model__max_depth': 13, 'model__bootstrap': True}
Best Score: -1.9860207069874143


### params for config : 

```python
pipeline = Pipeline(steps=[
    ('snowindexer', SnowIndexComputeTransformer(temp_col_name="tempartures_pca_1", rain_col_name="precipitations_pca_1",)),
    ('preprocessor', preprocessor),
    ('model', qrf_week1)
])

DATASET_TRANSFORMS = [
    "rm_gnv_st",
    "pca",
    # "snow_index",
    "oh_enc_date",
    "scl_wtr_flows",
    "scl_catch"
]
```


In [ ]:
print("Best Parameters:", random_search.best_params_)
print("Best Score:", random_search.best_score_)

Best Parameters: {'snowindexer__temp_weight': 1, 'snowindexer__precip_weight': 0.2, 'snowindexer__altitude_weight': 1, 'model__n_estimators': 52, 'model__min_samples_split': 19, 'model__min_samples_leaf': 13, 'model__max_features': None, 'model__max_depth': 25, 'model__bootstrap': True}
Best Score: -2.0340065806334726


### params for config : 
```python
DATASET_TRANSFORMS = [
    "rm_gnv_st",
    "pca",
    "snow_index",
    "oh_enc_date",
    "scl_wtr_flows",
    "scl_catch"
]
```


In [ ]:
print("Best Parameters:", random_search.best_params_)
print("Best Score:", random_search.best_score_)

Best Parameters: {'model__n_estimators': 55, 'model__min_samples_split': 20, 'model__min_samples_leaf': 11, 'model__max_features': None, 'model__max_depth': 23, 'model__bootstrap': True}
Best Score: -1.9197539988339372


### params for config : 
```python
DATASET_TRANSFORMS = [
    "rm_gnv_st",
    "pca",
    "snow_index",
    "oh_enc_date",
    "scl_wtr_flows",
    "scl_catch"
]
```


In [ ]:
print("Best Parameters:", random_search.best_params_)
print("Best Score:", random_search.best_score_)

Best Parameters: {'model__n_estimators': 60, 'model__min_samples_split': 20, 'model__min_samples_leaf': 9, 'model__max_features': None, 'model__max_depth': 13, 'model__bootstrap': True}
Best Score: -1.9379500224596236


### params for config : 
```python
DATASET_TRANSFORMS = [
    "rm_gnv_st",
    "pca",
    # "snow_index",
    "oh_enc_date",
    "scl_wtr_flows",
    "scl_catch"
]
```


In [ ]:
print("Best Parameters:", random_search.best_params_)
print("Best Score:", random_search.best_score_)

Best Parameters: {'model__n_estimators': 59, 'model__min_samples_split': 18, 'model__min_samples_leaf': 14, 'model__max_features': None, 'model__max_depth': 25, 'model__bootstrap': True}
Best Score: -1.923900362267943


### params for config : 
```python
DATASET_TRANSFORMS = [
    "rm_gnv_st",
    "pca",
    # "snow_index",
    "oh_enc_date",
    "scl_wtr_flows",
    "scl_catch"
]
```


In [ ]:
print("Best Parameters:", random_search.best_params_)
print("Best Score:", random_search.best_score_)

Best Parameters: {'model__n_estimators': 60, 'model__min_samples_split': 20, 'model__min_samples_leaf': 9, 'model__max_features': None, 'model__max_depth': 13, 'model__bootstrap': True}
Best Score: -1.9528121007193104


### params for config : 
```python
DATASET_TRANSFORMS = [
    "rm_gnv_st",
    "pca",
    # "snow_index",
    "oh_enc_date",
    "scl_wtr_flows",
    # "rm_st_id"
]
```


In [ ]:
print("Best Parameters:", random_search.best_params_)
print("Best Score:", random_search.best_score_)

Best Parameters: {'model__n_estimators': 57, 'model__min_samples_split': 20, 'model__min_samples_leaf': 14, 'model__max_features': None, 'model__max_depth': 22, 'model__bootstrap': True}
Best Score: -2.067613593686774


### params for config : 
```python
DATASET_TRANSFORMS = [
    "rm_gnv_st",
    "pca",
    # "snow_index",
    "oh_enc_date",
    "scl_wtr_flows",
    # "rm_st_id"
]
```


In [ ]:
# print("Best Parameters:", random_search.best_params_)
# print("Best Score:", random_search.best_score_)

Best Parameters: {'model__n_estimators': 55, 'model__min_samples_split': 22, 'model__min_samples_leaf': 16, 'model__max_features': None, 'model__max_depth': 25, 'model__bootstrap': True}
Best Score: -1.9144132951754593


### params for config : 
```python
DATASET_TRANSFORMS = [
    "rm_gnv_st",
    "pca",
    # "snow_index",
    "oh_enc_date",
    "scl_wtr_flows",
    # "rm_st_id"
]
```


In [ ]:
# print("Best Parameters:", random_search.best_params_)
# print("Best Score:", random_search.best_score_)

Best Parameters: {'model__n_estimators': 56, 'model__min_samples_split': 19, 'model__min_samples_leaf': 13, 'model__max_features': None, 'model__max_depth': 23, 'model__bootstrap': True}
Best Score: -1.9230515071418086


### params for config : 
```python
DATASET_TRANSFORMS = [
    "rm_gnv_st",
    "pca",
    # "snow_index",
    "oh_enc_date",
    "scl_wtr_flows",
    # "rm_st_id"
]
```


In [ ]:
# print("Best Parameters:", random_search.best_params_)
# print("Best Score:", random_search.best_score_)

Best Parameters: {'model__n_estimators': 57, 'model__min_samples_split': 20, 'model__min_samples_leaf': 11, 'model__max_features': None, 'model__max_depth': 17, 'model__bootstrap': True}
Best Score: -1.9198867119123673


### params for config : 
```python
DATASET_TRANSFORMS = [
    "rm_gnv_st",
    "pca",
    # "snow_index",
    "oh_enc_date",
    "scl_wtr_flows",
    # "rm_st_id"
]
```


In [ ]:
# print("Best Parameters:", random_search.best_params_)
# print("Best Score:", random_search.best_score_)

Best Parameters: {'model__n_estimators': 60, 'model__min_samples_split': 20, 'model__min_samples_leaf': 9, 'model__max_features': None, 'model__max_depth': 13, 'model__bootstrap': True}
Best Score: -2.1232930047642933


### params for config : 
```python
DATASET_TRANSFORMS = [
    "rm_gnv_st",
    "pca",
    # "snow_index",
    "oh_enc_date",
    "scl_wtr_flows",
    # "rm_st_id"
]
```


In [ ]:
# print("Best Parameters:", random_search.best_params_)
# print("Best Score:", random_search.best_score_)

Best Parameters: {'model__n_estimators': 60, 'model__min_samples_split': 20, 'model__min_samples_leaf': 9, 'model__max_features': None, 'model__max_depth': 13, 'model__bootstrap': True}
Best Score: -2.1232930047642933


### params for config : 
```python
DATASET_TRANSFORMS = [
    "remove_geneve_station",
    "full_pca",
    # "snow_index",
    "one_hot_encode_month_season",
    "scale_train_waterflows",
]
```


In [ ]:
# print("Best Parameters:", random_search.best_params_)
# print("Best Score:", random_search.best_score_)

Best Parameters: {'model__n_estimators': 60, 'model__min_samples_split': 20, 'model__min_samples_leaf': 9, 'model__max_features': None, 'model__max_depth': 13, 'model__bootstrap': True}
Best Score: -2.1232930047642933


### params for qrf pca_and_sin_season_encode

In [ ]:
# print("Best Parameters:", random_search.best_params_)
# print("Best Score:", random_search.best_score_)

Best Parameters: {'model__n_estimators': 60, 'model__min_samples_split': 20, 'model__min_samples_leaf': 9, 'model__max_features': None, 'model__max_depth': 13, 'model__bootstrap': True}
Best Score: -2.1232930047642933


### params for qrf full_pca

In [ ]:
# print("Best Parameters:", random_search.best_params_)
# print("Best Score:", random_search.best_score_)

Best Parameters: {'model__n_estimators': 60, 'model__min_samples_split': 20, 'model__min_samples_leaf': 9, 'model__max_features': None, 'model__max_depth': 13, 'model__bootstrap': True}
Best Score: -2.1232930047642933


#### b. GA

COMMING SOON